In [26]:
%pip install faiss-gpu



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os

# Khai bao bien
pdf_data_path = "data"
vector_db_path = "vectorstores/db_faiss"

def create_db_from_files():
    # Load all PDFs from the folder
    loader = DirectoryLoader(pdf_data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
    chunks = text_splitter.split_documents(documents)
    print('Done chunking')
    # Set up Gemini API key
    os.environ["GOOGLE_API_KEY"] = ""  # Thay bằng API key của bạn

    # Use Gemini embedding model
    embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

    # Create FAISS vector database (use from_documents, NOT from_texts)
    db = FAISS.from_documents(chunks, embedding_model)

    # Save database
    db.save_local(vector_db_path)
    return db

create_db_from_files()

e:\umbalaxibua\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

# Cấu hình API Key của Gemini
os.environ["GOOGLE_API_KEY"] = ""  # Thay bằng API key của bạn

# Đường dẫn database vector
vector_db_path = "vectorstores/db_faiss"

# Load LLM từ API Gemini
def load_llm():
    llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.01)
    return llm

# Tạo prompt template
def create_prompt():
    template = """Bạn là một trợ lý AI thông minh, được sử dụng để trả lời câu hỏi dựa trên dữ liệu cung cấp.
Dưới đây là ngữ cảnh từ tài liệu:
{context}

Câu hỏi: {question}

Hãy trả lời một cách đầy đủ và dễ hiểu bằng tiếng việt dựa trên thông tin có trong ngữ cảnh. Nếu không có thông tin phù hợp, hãy nói rằng bạn không biết.
"""
    prompt = PromptTemplate(template=template, input_variables=["context", "question"])
    return prompt

# Đọc dữ liệu từ VectorDB
def read_vectors_db():
    embedding_model = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    db = FAISS.load_local(vector_db_path, embedding_model, allow_dangerous_deserialization=True)

    return db

# Tạo chain truy vấn
def create_qa_chain(prompt, llm, db):
    llm_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 3}),
        return_source_documents=False,
        chain_type_kwargs={'prompt': prompt}
    )
    return llm_chain

# Khởi tạo các thành phần
db = read_vectors_db()
llm = load_llm()
prompt = create_prompt()
llm_chain = create_qa_chain(prompt, llm, db)

# Chạy chain với câu hỏi
question = "Why There Is No Documentation Phase"
response = llm_chain.invoke({"query": question})
print(response)


{'query': 'Why There Is No Documentation Phase', 'result': 'Không có giai đoạn lập tài liệu riêng biệt vì việc lập tài liệu là một quá trình liên tục và song hành với toàn bộ quá trình phát triển phần mềm.  Việc thiếu tài liệu đầy đủ và chính xác sẽ dẫn đến các vấn đề nghiêm trọng như:  không thể kiểm tra xem phần mềm có hoạt động đúng hay không và gần như không thể bảo trì phần mềm.  Tóm lại, lập tài liệu không phải là một giai đoạn riêng biệt mà là một phần không thể thiếu xuyên suốt quá trình phát triển phần mềm.'}
